In [1]:
article_list = []

for i in range(1, 511):
    if i < 10:
        i = '00' + str(i)
    elif i > 9 and i < 100:
        i = '0' + str(i)
    else:
        i = str(i)
        
    news_article = 'BBC_News_Summary/News_Articles/business/' + i + '.txt'
    news = ''
    with open(news_article) as f:
        for line in f.readlines():
            line = line.strip()
            news += line
            news += " "
        article_list.append(news)

In [2]:
article_list[6]

'Jobs growth still slow in the US  The US created fewer jobs than expected in January, but a fall in jobseekers pushed the unemployment rate to its lowest level in three years.  According to Labor Department figures, US firms added only 146,000 jobs in January. The gain in non-farm payrolls was below market expectations of 190,000 new jobs. Nevertheless it was enough to push down the unemployment rate to 5.2%, its lowest level since September 2001. The job gains mean that President Bush can celebrate - albeit by a very fine margin - a net growth in jobs in the US economy in his first term in office. He presided over a net fall in jobs up to last November\'s Presidential election - the first President to do so since Herbert Hoover. As a result, job creation became a key issue in last year\'s election. However, when adding December and January\'s figures, the administration\'s first term jobs record ended in positive territory.  The Labor Department also said it had revised down the jobs

In [3]:
summary_list = []

for i in range(1, 511):
    if i < 10:
        i = '00' + str(i)
    elif i > 9 and i < 100:
        i = '0' + str(i)
    else:
        i = str(i)
        
    news_article_summary = 'BBC_News_Summary/Summaries/business/' + i + '.txt'
    summary = '_START_ '
    
    with open(news_article_summary) as f:
        for line in f.readlines():
            line = line.strip()
            summary += line
            summary += " "
        summary = summary + " _END_"
        summary_list.append(summary)

In [4]:
summary_list[6]

'_START_ The job gains mean that President Bush can celebrate - albeit by a very fine margin - a net growth in jobs in the US economy in his first term in office.Analysts said the growth in new jobs was not as strong as could be expected given the favourable economic conditions.The Labor Department also said it had revised down the jobs gains in December 2004, from 157,000 to 133,000.The US created fewer jobs than expected in January, but a fall in jobseekers pushed the unemployment rate to its lowest level in three years."The economy is producing a moderate but not a satisfying amount of job growth," said Ken Mayland, president of ClearView Economics.He presided over a net fall in jobs up to last November\'s Presidential election - the first President to do so since Herbert Hoover.  _END_'

In [5]:
import pandas as pd

df = pd.DataFrame(columns = ['article', 'summary'])

for i in range(len(article_list)):
    row = pd.Series({'article' : article_list[i], 'summary' : summary_list[i]})
    df = pd.concat([df, row.to_frame().T], ignore_index = True)
    
df.head()

,article,summary
0,Ad sales boost Time Warner profit Quarterly p...,_START_ TimeWarner said fourth quarter sales r...
1,Dollar gains on Greenspan speech The dollar h...,_START_ The dollar has hit its highest level a...
2,Yukos unit buyer faces loan claim The owners ...,_START_ Yukos' owner Menatep Group says it wil...
3,High fuel prices hit BA's profits British Air...,"_START_ Rod Eddington, BA's chief executive, s..."
4,Pernod takeover talk lifts Domecq Shares in U...,_START_ Pernod has reduced the debt it took on...


In [6]:
from sklearn.model_selection import train_test_split
article_tr, article_val, summary_tr, summary_val=train_test_split(df['article'],
                                                                   df['summary'],
                                                                   test_size=0.1,
                                                                   random_state=0,
                                                                   shuffle=True) 

In [7]:
max_length = 0
articles = list(df['article'])

for article in articles:
    if len(article.split()) > max_length:
        max_length = len(article.split())
    else:
        continue

print(max_length)

891


In [8]:
max_length = 0
summaries = list(df['summary'])

for summary in summaries:
    if len(summary.split()) > max_length:
        max_length = len(summary.split())
    else:
        continue

print(max_length)

405


In [9]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

article_tokenizer = Tokenizer()
article_tokenizer.fit_on_texts(list(df['article']))

article_tr = article_tokenizer.texts_to_sequences(article_tr) 
article_val = article_tokenizer.texts_to_sequences(article_val) 

article_tr = pad_sequences(article_tr,  maxlen = 800, padding='post') 
article_val = pad_sequences(article_val, maxlen = 800, padding='post')

article_voc_size = len(article_tokenizer.word_index) +1

print(article_voc_size)

12675


In [10]:
summary_tokenizer = Tokenizer()
summary_tokenizer.fit_on_texts(list(df['summary']))

summary_tr = summary_tokenizer.texts_to_sequences(summary_tr) 
summary_val = summary_tokenizer.texts_to_sequences(summary_val) 

summary_tr = pad_sequences(summary_tr,  maxlen = 350, padding='post') 
summary_val = pad_sequences(summary_val, maxlen = 350, padding='post')

summary_voc_size = len(summary_tokenizer.word_index) +1

print(summary_voc_size)

8202


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Attention, Normalization, Dense
from tensorflow.keras.layers import Concatenate, TimeDistributed

In [32]:
embedding_inputs = Input(shape=(800), name = 'Encoder_Input') #max length of article
embedding = Embedding(12675, 3, trainable=True, name = 'Encoder_Embedding')(embedding_inputs) #vocab_size_article, neurons

encoder_layer_1 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_1') #neurons 
encoder_output_1, state_h1, state_c1 = encoder_layer_1(embedding)

encoder_layer_2 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_2') #neurons 
encoder_output_2, state_h2, state_c2 = encoder_layer_2(encoder_output_1)

encoder_layer_3 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_3') #neurons 
encoder_output_3, state_h3, state_c3 = encoder_layer_3(encoder_output_2)


decoder_inputs = Input(shape=(None,), name = 'Decoder_Input')
decoder_embedding = Embedding(8202, 50, trainable=True, name = 'Decoder_Embedding')(decoder_inputs)#vocab_size_summary, neurons

decoder_layer_1 = LSTM(32, return_sequences = True, return_state = True, name = 'Decoder_LSTM')
decoder_output_1, decoder_state_h1, decoder_state_c1 = decoder_layer_1(decoder_embedding, 
                                                                       initial_state=[state_h3, state_c3])

attention_layer = Attention()
attn_out, attn_state = attention_layer([encoder_output_3, decoder_output_1], return_attention_scores = True)


decoder_concat = Concatenate(axis=1)([decoder_output_1, attn_out])

decoder_dense = TimeDistributed(Dense(8202, activation = 'softmax'))#vocab_size_summary
decoder_outputs = decoder_dense(decoder_concat)

model = Model([embedding_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Encoder_Input (InputLayer)  [(None, 800)]                0         []                            
                                                                                                  
 Encoder_Embedding (Embeddi  (None, 800, 3)               38025     ['Encoder_Input[0][0]']       
 ng)                                                                                              
                                                                                                  
 LSTM_1 (LSTM)               [(None, 800, 32),            4608      ['Encoder_Embedding[0][0]']   
                              (None, 32),                                                         
                              (None, 32)]                                                   

In [33]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [16]:
labels = summary_tr.reshape(summary_tr.shape[0], summary_tr.shape[1], 1)[:,1:]
labels.shape

(459, 349, 1)

In [17]:
summary_tr[:,:-1].shape

(459, 349)

In [18]:
article_tr.shape

(459, 800)

In [34]:
history = model.fit([article_tr, summary_tr[:,:-1]], 
                    summary_tr.reshape(summary_tr.shape[0], summary_tr.shape[1], 1)[:,1:],
                    epochs=20,
                    batch_size=32)

Epoch 1/20


2023-07-05 14:47:01.903728: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/kawaii/opt/

In [ ]:
embedding_inputs = Input(shape=(800), name = 'Encoder_Input') #max length of article
embedding = Embedding(12675, 3, trainable=True, name = 'Encoder_Embedding')(embedding_inputs) #vocab_size_article, neurons

encoder_layer_1 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_1') #neurons 
encoder_output_1, state_h1, state_c1 = encoder_layer_1(embedding)

encoder_layer_2 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_2') #neurons 
encoder_output_2, state_h2, state_c2 = encoder_layer_2(encoder_output_1)

encoder_layer_3 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_3') #neurons 
encoder_output_3, state_h3, state_c3 = encoder_layer_3(encoder_output_2)


decoder_inputs = Input(shape=(None,), name = 'Decoder_Input')
decoder_embedding = Embedding(8202, 50, trainable=True, name = 'Decoder_Embedding')(decoder_inputs)#vocab_size_summary, neurons

decoder_layer_1 = LSTM(32, return_sequences = True, return_state = True, name = 'Decoder_LSTM')
decoder_output_1, decoder_state_h1, decoder_state_c1 = decoder_layer_1(decoder_embedding, 
                                                                       initial_state=[state_h3, state_c3])

decoder_dense = TimeDistributed(Dense(8202, activation = 'softmax'))#vocab_size_summary
decoder_outputs = decoder_dense(decoder_output_1)

model = Model([embedding_inputs, decoder_inputs], decoder_outputs)